In [1]:
#!ipcluster start -n 8 --engines=MPI --profile='mpi' # for parallel run: start the engines using terminal
from ipyparallel import Client
rc = Client(profile='mpi')

In [2]:
%%px
# Import the libraries

import os
import sys
import math
import pyJHTDB
import numpy as np
import pyfftw as ft 
from mpi4py import MPI
from pyJHTDB import libJHTDB
from pyJHTDB.dbinfo import isotropic1024coarse

from mpiFFT4py.slab import R2C

In [3]:
%%px

Nx = isotropic1024coarse['nx']; Ny = isotropic1024coarse['ny']; Nz = isotropic1024coarse['nz']
Lx = isotropic1024coarse['lx']; Ly = isotropic1024coarse['ly']; Lz = isotropic1024coarse['lz']
nu = isotropic1024coarse['nu']

comm = MPI.COMM_WORLD; rank = comm.Get_rank(); nproc = comm.Get_size()
if(rank==0):
    print("n_proc = "+str(nproc))
    print("rank = "+str(rank))

nx=Nx//nproc; ny=Ny; nz=Nz; nz_half=nz//2
nek=int(math.sqrt(2.0)/3*Nx); time = 0.0 

chkSz = 32; slabs = nx//chkSz

[stdout:7] 
n_proc = 8
rank = 0


In [4]:
%%px 

N = np.array([Nx,Ny,Nz],dtype=int)
L = np.array([Lx,Ly,Lz],dtype=float)

FFT = R2C(N, L, MPI.COMM_WORLD, "double", communication='Alltoallw')

In [5]:
%%px

if(rank==0):
    print(FFT.real_shape())
    print(FFT.complex_shape())

[stdout:7] 
(128, 1024, 1024)
(1024, 128, 513)


In [6]:
%%px

from DataDownload import DataDownload

# Populate velocity field from the Database

if(rank==0):
    print("Starting the loading process")

##########################################
load_from_file = True
store_on_file = False
auth_token = "com.gmail.jhelsas-b854269a"

folder = "/home/idies/workspace/scratch"
filename = "ref-isotropic1024coarse-"+str(rank)+"-(t="+str(time)+")"+".npz"
file = folder + "/" + filename

vx = np.zeros(FFT.real_shape(), dtype=FFT.float)
vy = np.zeros(FFT.real_shape(), dtype=FFT.float)
vz = np.zeros(FFT.real_shape(), dtype=FFT.float)
    
if(load_from_file):
    comm.Barrier(); t1=MPI.Wtime()
    content = np.load(file)
    if(int(content['nproc'])!=nproc):
        print("Unmatched number of processes. Must first pre-process to adequate number of process")
        
    vx[:,:,:] = content['vx'].astype(FFT.float)
    vy[:,:,:] = content['vy'].astype(FFT.float)
    vz[:,:,:] = content['vz'].astype(FFT.float)
    comm.Barrier(); t2=MPI.Wtime()
    if(rank==0):
        print("Finished loading")
        sys.stdout.write('Load from disk: {0:.2f} seconds\n'.format(t2-t1))
else:
    comm.Barrier(); t1=MPI.Wtime()
    
    ddwnld = DataDownload()
    vxd,vyd,vzd = ddwnld.DownldData_pyJHTDB('isotropic1024coarse',time,nx,ny,nz,nproc,rank,auth_token)
    
    vx[:,:,:] = vxd.astype(FFT.float)
    vy[:,:,:] = vyd.astype(FFT.float)
    vz[:,:,:] = vzd.astype(FFT.float)
    
    del vxd,vyd,vzd
    
    comm.Barrier(); t2=MPI.Wtime()
    if(rank==0):
        print("Finished loading")
        sys.stdout.write('Load field from database: {0:.2f} seconds\n'.format(t2-t1))
    
    comm.Barrier(); t1=MPI.Wtime()
    
    np.savez(file,vx=vx,vy=vy,vz=vz,nproc=nproc)
    comm.Barrier(); t2=MPI.Wtime()
    if(rank==0):
        sys.stdout.write('Caching the data: {0:.2f} seconds\n'.format(t2-t1))


if(rank==0):
    print("vx shape = "+str(vx.shape))       

[stdout:7] 
Starting the loading process
Finished loading
Load from disk: 274.17 seconds
vx shape = (128, 1024, 1024)


In [7]:
%%px
comm.Barrier(); t1=MPI.Wtime()

K = 0.5*(vx**2+vy**2+vz**2)

slabK = np.sum(K)

avgK=np.zeros(1,dtype=FFT.float)

comm.Reduce([slabK,MPI.DOUBLE],[avgK,MPI.DOUBLE],op=MPI.SUM)
avgK = avgK[0]*(1024**(-3))
avgK = comm.bcast(avgK, root=0)

comm.Barrier(); t2=MPI.Wtime()
if rank==0:
    print("kinectic energy = "+str(avgK))
    print("Computing kinectic energy: ", t2-t1)

[stdout:7] 
kinectic energy = 0.682719227202
Computing kinectic energy:  332.88546776771545


In [8]:
%%px
comm.Barrier(); t1=MPI.Wtime()

kx,ky,kz = FFT.get_local_wavenumbermesh()
k2 = kx**2 + ky**2 + kz**2

comm.Barrier(); t2=MPI.Wtime()
if(rank==0):
    print("Computing wavenumbers:",t2-t1)

[stdout:7] Computing wavenumbers: 83.57652306556702


In [9]:
%%px 
comm.Barrier(); t1=MPI.Wtime()

cvx = np.zeros(FFT.complex_shape(), dtype=FFT.complex)
cvy = np.zeros(FFT.complex_shape(), dtype=FFT.complex)
cvz = np.zeros(FFT.complex_shape(), dtype=FFT.complex)

comm.Barrier(); t2=MPI.Wtime()
if(rank==0):
    sys.stdout.write('Preparing FFT: {0:.2f} seconds\n'.format(t2-t1))

[stdout:7] Preparing FFT: 0.00 seconds


In [10]:
%%px
comm.Barrier(); t1=MPI.Wtime()

cvx = FFT.fftn(vx,cvx)
cvy = FFT.fftn(vy,cvy)
cvz = FFT.fftn(vz,cvz)

comm.Barrier(); t2=MPI.Wtime()
if(rank==0):
    sys.stdout.write('Calculate 3D spatial FFT: {0:.2f} seconds\n'.format(t2-t1))

[stdout:7] Calculate 3D spatial FFT: 658.37 seconds


In [11]:
%%px
comm.Barrier(); t1=MPI.Wtime()

cA = np.zeros(FFT.complex_shape(), dtype=FFT.complex)

comm.Barrier(); t2=MPI.Wtime()
if(rank==0):
    sys.stdout.write('alocating fourier space derivatives: {0:.2f} seconds\n'.format(t2-t1))

[stdout:7] alocating fourier space derivatives: 0.00 seconds


In [12]:
%%px
comm.Barrier(); t1=MPI.Wtime()

A11 = np.zeros(FFT.real_shape(), dtype=FFT.float)
A12 = np.zeros(FFT.real_shape(), dtype=FFT.float)
A13 = np.zeros(FFT.real_shape(), dtype=FFT.float)
A21 = np.zeros(FFT.real_shape(), dtype=FFT.float)
A22 = np.zeros(FFT.real_shape(), dtype=FFT.float)
A23 = np.zeros(FFT.real_shape(), dtype=FFT.float)
A31 = np.zeros(FFT.real_shape(), dtype=FFT.float)
A32 = np.zeros(FFT.real_shape(), dtype=FFT.float)
A33 = np.zeros(FFT.real_shape(), dtype=FFT.float)

comm.Barrier(); t2=MPI.Wtime()
if(rank==0):
    sys.stdout.write('Alocate real space gradients: {0:.2f} seconds\n'.format(t2-t1))

[stdout:7] Alocate real space gradients: 0.00 seconds


In [13]:
%%px
comm.Barrier(); t1=MPI.Wtime()
 
cA[:,:,:] = np.complex64(0.0+1.0j)*kx[:,:,:]*cvx[:,:,:]
A11 = FFT.ifftn(cA,A11)
A11[:,:,:] = A11[:,:,:]/(Nx*Ny*Nz)

cA[:,:,:] = np.complex64(0.0+1.0j)*kx[:,:,:]*cvy[:,:,:]
A12 = FFT.ifftn(cA,A12)
A12[:,:,:] = A12[:,:,:]/(Nx*Ny*Nz)

cA[:,:,:] = np.complex64(0.0+1.0j)*kx[:,:,:]*cvz[:,:,:]
A13 = FFT.ifftn(cA,A13)
A13[:,:,:] = A13[:,:,:]/(Nx*Ny*Nz)

comm.Barrier(); t2=MPI.Wtime()
if(rank==0):
    sys.stdout.write('Calculate velocity gradient in wavenumber space, and FFT back: {0:.2f} seconds\n'.format(t2-t1))

[stdout:7] Calculate velocity gradient in wavenumber space, and FFT back: 1430.46 seconds


In [14]:
%%px
comm.Barrier(); t1=MPI.Wtime()
 
cA[:,:,:] = np.complex64(0.0+1.0j)*ky[:,:,:]*cvx[:,:,:]
A21 = FFT.ifftn(cA,A21)
A21[:,:,:] = A21[:,:,:]/(Nx*Ny*Nz)

cA[:,:,:] = np.complex64(0.0+1.0j)*ky[:,:,:]*cvy[:,:,:]
A22 = FFT.ifftn(cA,A22)
A22[:,:,:] = A22[:,:,:]/(Nx*Ny*Nz)

cA[:,:,:] = np.complex64(0.0+1.0j)*ky[:,:,:]*cvz[:,:,:]
A23 = FFT.ifftn(cA,A23)
A23[:,:,:] = A23[:,:,:]/(Nx*Ny*Nz)

comm.Barrier(); t2=MPI.Wtime()
if(rank==0):
    sys.stdout.write('Calculate velocity gradient in wavenumber space: {0:.2f} seconds\n'.format(t2-t1))

[stdout:7] Calculate velocity gradient in wavenumber space: 2678.63 seconds


In [15]:
%%px
comm.Barrier(); t1=MPI.Wtime()
 
cA[:,:,:] = np.complex64(0.0+1.0j)*kz[:,:,:]*cvx[:,:,:]
A31 = FFT.ifftn(cA,A31)
A31[:,:,:] = A31[:,:,:]/(Nx*Ny*Nz)

cA[:,:,:] = np.complex64(0.0+1.0j)*kz[:,:,:]*cvy[:,:,:]
A32 = FFT.ifftn(cA,A32)
A32[:,:,:] = A32[:,:,:]/(Nx*Ny*Nz)

cA[:,:,:] = np.complex64(0.0+1.0j)*kz[:,:,:]*cvz[:,:,:]
A33 = FFT.ifftn(cA,A33)
A33[:,:,:] = A33[:,:,:]/(Nx*Ny*Nz)

comm.Barrier(); t2=MPI.Wtime()
if(rank==0):
    sys.stdout.write('Calculate velocity gradient in wavenumber space: {0:.2f} seconds\n'.format(t2-t1))

TypeError: %i format: a number is required, not NoneType

In [ ]:
%%px

del cA

In [ ]:
%%px

del cvx,cvy,cvz

In [18]:
%%px

del vx
del vy
del vz

CompositeError: one or more exceptions from call to method: execute
[0:execute]: NameError: name 'vx' is not defined
[1:execute]: NameError: name 'vx' is not defined
[2:execute]: NameError: name 'vx' is not defined
[3:execute]: NameError: name 'vx' is not defined
.... 4 more exceptions ...

In [20]:
%%px 

del cvx,cvy,cvz

CompositeError: one or more exceptions from call to method: execute
[0:execute]: NameError: name 'cvx' is not defined
[1:execute]: NameError: name 'cvx' is not defined
[2:execute]: NameError: name 'cvx' is not defined
[3:execute]: NameError: name 'cvx' is not defined
.... 4 more exceptions ...

In [ ]:
%%px 

A  = [A11,A12,A13,A21,A22,A23,A31,A32,A33]
la = ['A11','A12','A13','A21','A22','A23','A31','A32','A33']

for i in range(9):
    Aav = np.average(A[i])
    slabAavg = np.sum(Aav)
    
    avgA =np.zeros(1,dtype='float32')
    
    comm.Reduce([slabAavg,MPI.REAL],[avgA,MPI.REAL],op=MPI.SUM)
    avgA = avgA[0]/nproc #avgK[0]*(1024**(-3))
    avgA = comm.bcast(avgA, root=0)
    
    if rank==0:
        print("<"+la[i]+"> = "+str(avgA))

In [ ]:
%%px 

A  = [A11,A12,A13,A21,A22,A23,A31,A32,A33]
la = ['A11','A12','A13','A21','A22','A23','A31','A32','A33']

for i in range(9):
    A2av = np.average(A[i]**2)
    
    avgA2 =np.zeros(1,dtype='float32')
    
    comm.Reduce([A2av,MPI.REAL],[avgA2,MPI.REAL],op=MPI.SUM)
    avgA2 = avgA2[0]/nproc #avgK[0]*(1024**(-3))
    avgA2 = comm.bcast(avgA2, root=0)
    
    if rank==0:
        print("<"+la[i]+"^2> = "+str(avgA2))

In [ ]:
%%px 

A  = [A11,A12,A13,A21,A22,A23,A31,A32,A33]
la = ['A11','A12','A13','A21','A22','A23','A31','A32','A33']

for i in range(9):  
    A2av = np.average(A[i]**2)
    A3av = np.average(A[i]**3)
    
    avgA2 =np.zeros(1,dtype='float32')
    avgA3 =np.zeros(1,dtype='float32')
    
    comm.Reduce([A2av,MPI.REAL],[avgA2,MPI.REAL],op=MPI.SUM)
    comm.Reduce([A3av,MPI.REAL],[avgA3,MPI.REAL],op=MPI.SUM)
    
    avgA2 = avgA2[0]/nproc 
    avgA2 = comm.bcast(avgA2, root=0)
    avgA3 = avgA3[0]/nproc 
    avgA3 = comm.bcast(avgA3, root=0)
    
    if rank==0:
        print("skewness = <"+la[i]+"^3>/"+"<"+la[i]+"^2>^(3/2) "+"= "+str(avgA3/(avgA2**(3/2))) )

In [ ]:
%%px 

div = A11+A22+A33 

div2av = np.average(div**2)
    
avgdiv2 =np.zeros(1,dtype='float32')
   
comm.Reduce([div2av,MPI.REAL],[avgdiv2,MPI.REAL],op=MPI.SUM)
avgdiv2 = avgdiv2[0]/nproc 
avgdiv2 = comm.bcast(avgdiv2, root=0)

if rank==0:
    print("<(div v)^2> = "+str(avgdiv2))

In [ ]:
%%px

vx_xlp = vx[:,0,0]

vx_xl = comm.gather(vx_xlp,root=0)
if rank==0:
    l = tuple(vx_xl)
    vx_xl = np.concatenate(l)

vx_yl = vx[0,:,0]
vx_zl = vx[0,0,:]

##########################

vy_xlp = vy[:,0,0]

vy_xl = comm.gather(vy_xlp,root=0)
if rank==0:
    l = tuple(vy_xl)
    vy_xl = np.concatenate(l)

vy_yl = vy[0,:,0]
vy_zl = vy[0,0,:]

##########################

vz_xlp = vz[:,0,0]

vz_xl = comm.gather(vz_xlp,root=0)
if rank==0:
    l = tuple(vz_xl)
    vz_xl = np.concatenate(l)

vz_yl = vz[0,:,0]
vz_zl = vz[0,0,:]

if rank==0:
    print(vx_xl.shape)
    print(vx_yl.shape)
    print(vx_zl.shape)

In [ ]:
%%px

vx_xlpdx = A11[:,0,0]

vx_xldx = comm.gather(vx_xlpdx,root=0)
if rank==0:
    ldx = tuple(vx_xldx)
    vx_xldx = np.concatenate(ldx)

vx_yldx = A21[0,:,0]
vx_zldx = A31[0,0,:]

########################

vy_xlpdx = A12[:,0,0]

vy_xldx = comm.gather(vy_xlpdx,root=0)
if rank==0:
    ldy = tuple(vy_xldx)
    vy_xldx = np.concatenate(ldy)

vy_yldx = A22[0,:,0]
vy_zldx = A32[0,0,:]

########################


vz_xlpdx = A13[:,0,0]

vz_xldx = comm.gather(vz_xlpdx,root=0)
if rank==0:
    ldz = tuple(vz_xldx)
    vz_xldx = np.concatenate(ldz)

vz_yldx = A23[0,:,0]
vz_zldx = A33[0,0,:]

In [ ]:
%%px
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

Y = np.linspace(0,2.*np.pi,Ny)
print(Y.shape)

if rank==0:
    plt.plot(Y,vx_xl,'r-')
    plt.plot(Y,vx_xldx/100,'b-')
    plt.grid()
    
    plt.xlim((0,2))
    
    plt.show()

In [ ]:
%%px 

if rank==0:
    pos = vx_xldx > 0
    neg = vx_xldx < 0
    scale = 30
    
    dx = 2.*np.pi/1024
    vlgrx = np.gradient(vx_xl)/dx
    
    plt.plot(Y[pos],vx_xl[pos],'r.')
    plt.plot(Y[pos],vx_xldx[pos]/scale,'b.')
    
    plt.plot(Y[neg],vx_xl[neg],'m.')
    plt.plot(Y[neg],vx_xldx[neg]/scale,'k.')
    
    plt.plot(Y,vlgrx/scale,'g')
    plt.grid()
    
    plt.xlim((1,2))
    
    plt.show()

In [ ]:
%%px 

if rank==0:
    pos = vx_xldx > 0
    neg = vx_xldx < 0
    scale = 30
    
    dx = 2.*np.pi/1024
    vxlgrx = np.gradient(vx_xl)/dx
    vylgrx = np.gradient(vy_xl)/dx
    vzlgrx = np.gradient(vz_xl)/dx
            
    plt.xlim((1,2))
    plt.grid()
    plt.plot(Y,vx_xldx,'r.')
    plt.plot(Y,vxlgrx,'c-')
    
    plt.show()
    
    plt.xlim((1,2))
    plt.grid()
    plt.plot(Y,vy_xldx,'g.')
    plt.plot(Y,vylgrx,'k-')
    
    plt.show()
    
    plt.xlim((1,2))
    plt.grid()
    plt.plot(Y,vz_xldx,'b.')
    plt.plot(Y,vzlgrx,'m-')
    
    plt.show()

In [ ]:
%%px 

if rank==0:
    pos = vx_xldx > 0
    neg = vx_xldx < 0
    scale = 30
    
    dx = 2.*np.pi/1024
    vxlgrx = np.gradient(vx_xl)/dx
    vxlgry = np.gradient(vx_yl)/dx
    vxlgrz = np.gradient(vx_zl)/dx
            
    plt.xlim((1,2))
    plt.grid()
    plt.plot(Y,vx_xldx,'r.')
    plt.plot(Y,vxlgrx,'c-')
    
    plt.show()
    
    plt.xlim((1,2))
    plt.grid()
    plt.plot(Y,vx_yldx,'g.')
    plt.plot(Y,vxlgry,'k-')
    
    plt.show()
    
    plt.xlim((1,2))
    plt.grid()
    plt.plot(Y,vx_yldx,'b.')
    plt.plot(Y,vxlgry,'m-')
    
    plt.show()

In [ ]:
%%px 

if rank==0:
    pos = vx_xldx > 0
    neg = vx_xldx < 0
    scale = 30
    
    dx = 2.*np.pi/1024
    vylgrx = np.gradient(vy_xl)/dx
    vylgry = np.gradient(vy_yl)/dx
    vylgrz = np.gradient(vy_zl)/dx
            
    plt.xlim((1,2))
    plt.grid()
    plt.plot(Y,vy_xldx,'r.')
    plt.plot(Y,vylgrx,'c-')
    
    plt.show()
    
    plt.xlim((1,2))
    plt.grid()
    plt.plot(Y,vy_yldx,'g.')
    plt.plot(Y,vylgry,'k-')
    
    plt.show()
    
    plt.xlim((1,2))
    plt.grid()
    plt.plot(Y,vy_zldx,'b.')
    plt.plot(Y,vylgrz,'m-')
    
    plt.show()

In [ ]:
%%px 

Q = 0.5*( (A12*A21+A23*A32+A13*A31) - (A22*A33+A11*A33+A11*A22) )

In [ ]:
%%px

slabQ = np.average(Q)

avgQ=np.zeros(1,dtype='float32')

comm.Reduce([slabQ,MPI.REAL],[avgQ,MPI.REAL],op=MPI.SUM)
avgQ = avgQ[0]/nproc
avgQ = comm.bcast(avgQ, root=0)

if rank==0:
    print("Average Q = "+str(avgQ))

In [ ]:
%%px

R = A11**3 + A22**3 + A33**3 + 3*( A11*(A21*A12+A13*A31) + A22*(A12*A21+A23*A32) + A33*(A13*A31+A23*A32) + A13*A21*A32 + A12*A23*A31 )

In [ ]:
%%px

slabR = np.average(R)

avgR=np.zeros(1,dtype='float32')

comm.Reduce([slabR,MPI.REAL],[avgR,MPI.REAL],op=MPI.SUM)
avgR = avgR[0]/nproc
avgR = comm.bcast(avgR, root=0)

if rank==0:
    print("Average R = "+str(avgR))

In [ ]:
%%px

slabQ2 = np.average(Q**2)

avgQ2=np.zeros(1,dtype='float32')

comm.Reduce([slabQ2,MPI.REAL],[avgQ2,MPI.REAL],op=MPI.SUM)
avgQ2 = avgQ2[0]/nproc
avgQ2 = comm.bcast(avgQ2, root=0)

if rank==0:
    print("Average Q^2 = "+str(avgQ2))

In [ ]:
%%px

slabR = np.average(R)

avgR=np.zeros(1,dtype='float32')

comm.Reduce([slabR,MPI.REAL],[avgR,MPI.REAL],op=MPI.SUM)
avgR = avgR[0]/nproc
avgR = comm.bcast(avgR, root=0)

if rank==0:
    print("<R>/(<Q^2>^3/2) = "+str(avgR/(avgQ2**1.5)))

In [ ]:
%%px 

del Q,R

In [ ]:
%%px

Omega = 0.5*( A12**2 + A21**2 + A13**2 + A31**2 + A23**2 + A32**2 - 2.*(A12*A21+A13*A31+A23*A32) )

In [ ]:
%%px

slabO = np.average(Omega)

avgO=np.zeros(1,dtype='float32')

comm.Reduce([slabO,MPI.REAL],[avgO,MPI.REAL],op=MPI.SUM)
avgO = avgO[0]/nproc
avgO = comm.bcast(avgO, root=0)

if rank==0:
    print("Average enstrophy = "+str(avgO))

In [ ]:
%%px

Epsilon  = A11**2 + A22**2 + A33**2 
Epsilon += 0.5*( A12**2 + A21**2 + A23**2 + A32**2 + A13**2 + A31**2 )
Epsilon += A12*A21 + A13*A31 + A23*A32

In [ ]:
%%px

slabE = np.average(Epsilon)

avgE=np.zeros(1,dtype='float32')

comm.Reduce([slabE,MPI.REAL],[avgE,MPI.REAL],op=MPI.SUM)
avgE = avgE[0]/nproc
avgE = comm.bcast(avgE, root=0)

if rank==0:
    print("Average strainrate = "+str(avgE))

In [ ]:
%%px

if rank==0:
    print("Average strainrate - enstrophy: "+str(avgE-avgO))
    print("Normalized strainrate - enstrophy: "+str((avgE-avgO)/avgO))

In [ ]:
%%px

del Omega, Epsilon

In [ ]:
%%px

eps = 2*nu*avgE

In [ ]:
%%px

urms = np.sqrt(2.*avgK/3)
lamb = np.sqrt(15.*nu/eps)*urms
ReTayor = urms*lamb/nu
if rank==0:
    print("nu = "+str(nu))
    print("urms = "+str(urms)+", Reference: 0.6820399")
    print("eps = "+ str(eps))
    print("kinectic energy = "+str(avgK))
    print("Taylor micro-scale ="+str(lamb))
    print("ReLamb = "+str(ReTayor)+", Reference: 426.8378")